In [1]:
import pandas as pd

In [2]:
unique = pd.read_csv('unique_user_location.csv')


In [3]:
len(unique)

886

In [4]:
unique.head()

,# favorite of tweet,# followers,# following,# of hastags,# of mentions,account creation date,in reply to,location,name,number of account tweets,...,race,screen name,time,tweet,user id,classification,compound score,Racist (Y/N),State Abbreviation,Valid City Data (Y/N)
0,0,1035,695,0,0,Mon May 10 20:46:10 +0000 2010,NaN,"Los Angeles, CA",Daunt L. Guapo,46837,...,black,TheAntiThirst,Sun Sep 24 23:43:48 +0000 2017,"""Black Men are the White People of Black Peopl...",1.424148e+08,Hate speech,0.0000,y,CA,y
1,0,34,80,0,2,Mon Jun 06 20:29:37 +0000 2016,7.990000e+17,"Tennessee, USA",Donald J Trump,109,...,black,potus45ofUSA,Sun Sep 24 23:41:03 +0000 2017,"@KathySc65174870 @teacherfortrump Kathy, it wa...",7.400000e+17,Hate speech,-0.3818,y,TN,y
2,1,3007,5000,0,3,Sun May 21 01:57:26 +0000 2017,7.810000e+17,"Dallas, TX",Kristin Fennymore,5069,...,black,KFennymore,Sun Sep 24 23:40:02 +0000 2017,@erick_c_h01 @VeronicaA5567 @Trumpfan1995 I'm ...,8.660000e+17,Hate speech,-0.6808,y,TX,y
3,1,3,0,1,0,Mon Aug 07 00:08:24 +0000 2017,NaN,"Simi Valley, CA",Merica1958,456,...,black,Merica1861,Sun Sep 24 23:18:48 +0000 2017,#TakeTheKnee Black guy shoots up a church of w...,8.940000e+17,Hate speech,-0.7579,y,CA,y
4,0,2321,1198,0,0,Thu Aug 12 03:04:35 +0000 2010,NaN,"East Flatbush, Brooklyn",Nola Darling????,257313,...,black,WESTCOASTSUMA,Sun Sep 24 23:15:47 +0000 2017,This is why people say black men are the white...,1.774275e+08,Hate speech,0.0000,y,NY,y


In [11]:
unique['race'].value_counts()

black     354
jews      290
muslim    242
Name: race, dtype: int64

In [21]:
grouped = pd.DataFrame(unique.groupby(['race', 'State Abbreviation'])['race'].count())

In [25]:
grouped1 = grouped.unstack(level = 0)
grouped1.columns = grouped1.columns.get_level_values(1)

In [27]:
grouped1 = grouped1.fillna(0)

In [48]:
population = pd.read_csv('census_data/2016_population.csv')

In [49]:
population.rename(columns = {'Unnamed: 0': 'State'}, inplace = True)

In [50]:
population = population.set_index('State')


In [54]:
population = pd.DataFrame(population['Total Population']).reset_index()

In [55]:
population.head()

,State,Total Population
0,Alabama,4863300
1,Alaska,741894
2,Arizona,6931071
3,Arkansas,2988248
4,California,39250017


In [62]:
# State abbreviation dictionary
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    'District of Columbia': 'DC'
}

In [58]:
population['State Abbr'] = ""

In [65]:
for index, row in population.iterrows():
    state_abbr = us_state_abbrev[row['State']]
    population.set_value(index, 'State Abbr', state_abbr)
    
    

In [69]:
grouped1.head()

race,black,jews,muslim
State Abbreviation,,,
AK,1.0,0.0,1.0
AL,1.0,1.0,4.0
AR,3.0,1.0,3.0
AZ,1.0,0.0,4.0
CA,30.0,31.0,17.0


In [72]:
population = population.merge(grouped1, left_on = 'State Abbr', right_index = True)

In [81]:
population['norm_black'] = population['black']/population['Total Population'] * 100
population['norm_jewish'] = population['jews']/population['Total Population'] * 100
population['norm_muslim'] = population['muslim']/population['Total Population'] * 100
population['total_count'] = population['black'] + population['jews'] + population['muslim']
population['total_norm'] = population['total_count']/population['Total Population'] * 100

In [82]:
population.head()

,State,Total Population,State Abbr,black,jews,muslim,norm_black,norm_jewish,norm_muslim,total_count,total_norm
0,Alabama,4863300,AL,1.0,1.0,4.0,0.000021,0.000021,0.000082,6.0,0.000123
1,Alaska,741894,AK,1.0,0.0,1.0,0.000135,0.000000,0.000135,2.0,0.000270
2,Arizona,6931071,AZ,1.0,0.0,4.0,0.000014,0.000000,0.000058,5.0,0.000072
3,Arkansas,2988248,AR,3.0,1.0,3.0,0.000100,0.000033,0.000100,7.0,0.000234
4,California,39250017,CA,30.0,31.0,17.0,0.000076,0.000079,0.000043,78.0,0.000199
